In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from Environment import Environment
from Agent import Agent
from LinearRegression import LinearRegression
from TicTacToe import TicTacToe
import pickle

In [ ]:
model = LinearRegression(7)
for game in tqdm(range(10000)):
    env = Environment()
    ag1 = Agent('x', model)
    ag2 = Agent('o', model)
    agents = [ag1, ag2]
    features = [[], []]
    turn = 0
    while True:
        curr_board = env.get_state()
        action, feature = agents[turn].predict_action(curr_board)
        features[turn].append(feature)
        env.make_action(action)
        stat = env.finish()
        turn = 1 - turn
        if stat > 0:
            break

    if stat == 1:
        data = np.stack(features[0] + features[1])
        labels = np.zeros((len(data), 1))
    else:
        winner = stat % 2
        data = np.stack(features[winner] + features[1-winner])
        labels = np.concatenate([np.ones((len(features[winner]), 1))*100,np.ones((len(features[1-winner]), 1))*-100], axis = 0)

    ind = np.arange(len(data))
    np.random.shuffle(ind)
    data = data[ind]
    labels = labels[ind]

    model.fit(data, labels)

In [ ]:
with open('linear_regression_model.pkl', 'wb') as file:
    pickle.dump(model.params, file)